# Modelling metabolism in MAIZE

### Generating MAIZE core model

In [1]:
from cobra import io
from cobra.core import Metabolite, Reaction
from cobra import flux_analysis
from Functions import *
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "/home/sanu/Documents/Scripts/git/plantcoremetabolism-model/PlantCoreMetabolism_v1_3_0.xml"
model = io.sbml.read_sbml_model(fname)
display.clear_output()

from sweetlovegroup.transform import fixModelCompatibilityIssueCobra015
model = fixModelCompatibilityIssueCobra015(model,fname)


#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
met.compartment = "b"
rxn.add_metabolites({met:1})


for rxn in model.reactions:
    if rxn.lower_bound == -1000:
        rxn.lower_bound = -3000
    if rxn.upper_bound == 1000:
        rxn.upper_bound = 3000

### Use biomass composition to generate biomass equations

In [4]:
def generateMissingFormula(model):
    loop_counter = 0
    former = 0
    for met in model.metabolites:
        if met.formula == "" or met.formula == "NA":
            former = former +1
    latter = 1
    while True:
        loop_counter = loop_counter+1
        print "Loop = "+str(loop_counter)
        former = latter
        for rxn in model.reactions:
            count = 0
            for met in rxn.metabolites:
                if met.formula=="" or met.formula=="NA" or met.formula == None:
                    if met.formula == "NA" or met.formula == None:
                        met.formula = ""
                    count = count + 1
            if count == 1:
                unb = rxn.check_mass_balance()
                for met in rxn.metabolites:
                    if met.formula == "":
                        tempForm = ""
                        for a in unb.keys():
                            if a=="charge" or round(unb[a],2)==0:
                                continue
                            tempForm = tempForm+a+str(abs(unb[a]))
                        met.formula = tempForm
                        print met.id
                        print tempForm
        latter = 0
        for met in model.metabolites:
            if met.formula == "" or met.formula == "NA":
                latter = latter +1
        if former == latter:
            break
        
        

In [5]:
import pandas as pd

df = pd.read_csv("Data/biomass_maize.csv")

FA=["PALMITATE_p","CPD_9245_p","CPD_17412_p","CPD_17291_p","STEARIC_ACID_p","OLEATE_CPD_p",
    "Octadecadienoate_p","LINOLENIC_ACID_p","ARACHIDIC_ACID_p","CPD_16709_p","DOCOSANOATE_p"]
FACP = {"PALMITATE_p":"Palmitoyl_ACPs_p",
        "CPD_9245_p":"Palmitoleoyl_ACP_p",
        "CPD_17412_p":"hexadecadienoate_ACP_p",
        "CPD_17291_p":"hexadecatrienoate_ACP_p",
        "STEARIC_ACID_p":"Stearoyl_ACPs_p",
        "OLEATE_CPD_p":"Oleoyl_ACPs_p",
        "Octadecadienoate_p":"Octadecadienoyl_ACP_p",
        "LINOLENIC_ACID_p":"Octadecatrienoyl_ACP_p",
        "ARACHIDIC_ACID_p":"Arachidoyl_ACPs_p",
        "CPD_16709_p":"Eicosenoyl_ACP_p",
        "DOCOSANOATE_p":"Behenoyl_ACPs_p"}



### Run stem

In [6]:
from cobra import flux_analysis
stem_model = model.copy()

k = "stem"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({stem_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])/stem_model.metabolites.get_by_id(met).formula_weight})
        RXN2.add_metabolites({stem_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])/stem_model.metabolites.get_by_id(met).formula_weight})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k])/stem_model.metabolites.get_by_id(met).formula_weight)
if tot==0:
    RXN1.add_metabolites({stem_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({stem_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({stem_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 0
stem_model.add_reaction(RXN1)

RXN2.add_metabolites({stem_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 0
stem_model.add_reaction(RXN2)


generateMissingFormula(stem_model)

rxn = Reaction("Biomass_stem_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({stem_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met]["stem"])/stem_model.metabolites.get_by_id(met).formula_weight})
rxn.lower_bound = 0
rxn.upper_bound = 1000
stem_model.add_reaction(rxn)


stem_model.reactions.GLC_tx.upper_bound = 0
stem_model.reactions.GLC_tx.lower_bound = 0
stem_model.reactions.Sucrose_tx.upper_bound = 1
stem_model.reactions.Sucrose_tx.lower_bound = 1
stem_model.reactions.Photon_tx.upper_bound = 0
stem_model.reactions.Photon_tx.lower_bound = 0
stem_model.reactions.Biomass_stem_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(stem_model)
print("Biomass flux ="+str(stem_model.reactions.Biomass_stem_tx.flux))

Loop = 1
Photon_e

Fatty_acyl_ACP_p
H31.0C16.0O1.0X1.0
Protein_polymerisation_cost_c

Protein_processing_cost_c

Photon_p

Palmitate_b
H31.0C16.0O2.0
Photon_p

Protein_tranlocation_cost_c

Long_Chain_Acyl_CoAs_p
C37.0H66.0O17.0N7.0P3.0S1.0
Fatty_Acids_c
H31.0C16.0O2.0
Fatty_Acids_p
H31.0C16.0O2.0
3_oxo_stearoyl_ACPs_p
H33.0C18.0O2.0X1.0
R_3_hydroxystearoyl_ACPs_p
H35.0C18.0O2.0X1.0
Octadec_2_enoyl_ACPs_p
H33.0C18.0O1.0X1.0
Stearoyl_ACPs_p
H33.0C18.0X1.0
Palmitoleoyl_ACP_p
X1.0C16.0O1.0H31.0
hexadecadienoate_ACP_p
X1.0C16.0O1.0H31.0
hexadecatrienoate_ACP_p
X1.0C16.0O1.0H31.0
Oleoyl_ACPs_p
X1.0C18.0H33.0
Octadecadienoyl_ACP_p
X1.0C18.0H33.0
Octadecatrienoyl_ACP_p
X1.0C18.0H33.0
3_oxo_arachidoyl_ACPs_p
H35.0C20.0O1.0X1.0
R_3_hydroxyarachidoyl_ACPs_p
H37.0C20.0O1.0X1.0
trans_delta2_arachidoyl_ACPs_p
H35.0C20.0X1.0
Arachidoyl_ACPs_p
H35.0C20.0O1.0X1.0
3_oxo_behenoyl_ACPs_p
H37.0C22.0O2.0X1.0
R_3_hydroxybehenoyl_ACPs_p
H39.0C22.0O2.0X1.0
trans_delta2_behenoyl_ACPs_p
H37.0C22.0O1.0X1.0
Beheno

cobra/core/metabolite.py:104 UserWarning: 43.03 is not an integer (in formula H43.03C22.0O12.0P2.0)
cobra/core/metabolite.py:104 UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 25.9492553613 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 5.7497158698 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 4.9536245179 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)


### Run root

In [7]:
from cobra import flux_analysis
root_model = model.copy()

k = "root"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({root_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])/root_model.metabolites.get_by_id(met).formula_weight})
        RXN2.add_metabolites({root_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])/root_model.metabolites.get_by_id(met).formula_weight})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k])/root_model.metabolites.get_by_id(met).formula_weight)
if tot==0:
    RXN1.add_metabolites({root_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({root_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({root_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 0
root_model.add_reaction(RXN1)
    
RXN2.add_metabolites({root_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 0
root_model.add_reaction(RXN2)


generateMissingFormula(root_model)

rxn = Reaction("Biomass_root_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({root_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met]["stem"])/root_model.metabolites.get_by_id(met).formula_weight})
rxn.lower_bound = 0
rxn.upper_bound = 1000
root_model.add_reaction(rxn)

root_model.reactions.GLC_tx.upper_bound = 0
root_model.reactions.GLC_tx.lower_bound = 0
root_model.reactions.Sucrose_tx.upper_bound = 1
root_model.reactions.Sucrose_tx.lower_bound = 1
root_model.reactions.Photon_tx.upper_bound = 0
root_model.reactions.Photon_tx.lower_bound = 0
root_model.reactions.Biomass_root_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(root_model)
print("Biomass flux ="+str(root_model.reactions.Biomass_root_tx.flux))

Loop = 1
Photon_e

Fatty_acyl_ACP_p
H31.0C16.0O1.0X1.0
Protein_polymerisation_cost_c

Protein_processing_cost_c

Photon_p

Palmitate_b
H31.0C16.0O2.0
Photon_p

Protein_tranlocation_cost_c

Long_Chain_Acyl_CoAs_p
C37.0H66.0O17.0N7.0P3.0S1.0
Fatty_Acids_c
H31.0C16.0O2.0
Fatty_Acids_p
H31.0C16.0O2.0
3_oxo_stearoyl_ACPs_p
H33.0C18.0O2.0X1.0
R_3_hydroxystearoyl_ACPs_p
H35.0C18.0O2.0X1.0
Octadec_2_enoyl_ACPs_p
H33.0C18.0O1.0X1.0
Stearoyl_ACPs_p
H33.0C18.0X1.0
Palmitoleoyl_ACP_p
H31.0C16.0O1.0X1.0
hexadecadienoate_ACP_p
X1.0C16.0O1.0H31.0
hexadecatrienoate_ACP_p
X1.0C16.0O1.0H31.0
Oleoyl_ACPs_p
H33.0C18.0X1.0
Octadecadienoyl_ACP_p
X1.0C18.0H33.0
Octadecatrienoyl_ACP_p
X1.0C18.0H33.0
3_oxo_arachidoyl_ACPs_p
H35.0C20.0O1.0X1.0
R_3_hydroxyarachidoyl_ACPs_p
H37.0C20.0O1.0X1.0
trans_delta2_arachidoyl_ACPs_p
H35.0C20.0X1.0
Arachidoyl_ACPs_p
H35.0C20.0O1.0X1.0
3_oxo_behenoyl_ACPs_p
H37.0C22.0O2.0X1.0
R_3_hydroxybehenoyl_ACPs_p
H39.0C22.0O2.0X1.0
trans_delta2_behenoyl_ACPs_p
H37.0C22.0O1.0X1.0
Beheno

### Run seed

In [8]:
from cobra import flux_analysis
seed_model = model.copy()

k = "seed"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({seed_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])/seed_model.metabolites.get_by_id(met).formula_weight})
        RXN2.add_metabolites({seed_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])/seed_model.metabolites.get_by_id(met).formula_weight})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k])/seed_model.metabolites.get_by_id(met).formula_weight)
if tot==0:
    RXN1.add_metabolites({seed_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({seed_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({seed_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 0
seed_model.add_reaction(RXN1)
    
RXN2.add_metabolites({seed_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 0
seed_model.add_reaction(RXN2)


generateMissingFormula(seed_model)

rxn = Reaction("Biomass_seed_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({seed_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met]["stem"])/seed_model.metabolites.get_by_id(met).formula_weight})
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)

seed_model.reactions.GLC_tx.upper_bound = 0
seed_model.reactions.GLC_tx.lower_bound = 0
seed_model.reactions.Sucrose_tx.upper_bound = 1
seed_model.reactions.Sucrose_tx.lower_bound = 1
seed_model.reactions.Photon_tx.upper_bound = 0
seed_model.reactions.Photon_tx.lower_bound = 0
seed_model.reactions.Biomass_seed_tx.objective_coefficient=1
sol=flux_analysis.parsimonious.pfba(seed_model)
print("Biomass flux ="+str(seed_model.reactions.Biomass_seed_tx.flux))

    
    

Loop = 1
Photon_e

Fatty_acyl_ACP_p
H31.0C16.0O1.0X1.0
Protein_polymerisation_cost_c

Protein_processing_cost_c

Photon_p

Palmitate_b
H31.0C16.0O2.0
Photon_p

Protein_tranlocation_cost_c

Long_Chain_Acyl_CoAs_p
C37.0H66.0O17.0N7.0P3.0S1.0
Fatty_Acids_c
H31.0C16.0O2.0
Fatty_Acids_p
H31.0C16.0O2.0
3_oxo_stearoyl_ACPs_p
H33.0C18.0O2.0X1.0
R_3_hydroxystearoyl_ACPs_p
H35.0C18.0O2.0X1.0
Octadec_2_enoyl_ACPs_p
H33.0C18.0O1.0X1.0
Stearoyl_ACPs_p
H33.0C18.0X1.0
Palmitoleoyl_ACP_p
X1.0C16.0O1.0H31.0
hexadecadienoate_ACP_p
X1.0C16.0O1.0H31.0
hexadecatrienoate_ACP_p
X1.0C16.0O1.0H31.0
Oleoyl_ACPs_p
X1.0C18.0H33.0
Octadecadienoyl_ACP_p
X1.0C18.0H33.0
Octadecatrienoyl_ACP_p
X1.0C18.0H33.0
3_oxo_arachidoyl_ACPs_p
H35.0C20.0O1.0X1.0
R_3_hydroxyarachidoyl_ACPs_p
H37.0C20.0O1.0X1.0
trans_delta2_arachidoyl_ACPs_p
H35.0C20.0X1.0
Arachidoyl_ACPs_p
H35.0C20.0O1.0X1.0
3_oxo_behenoyl_ACPs_p
H37.0C22.0O2.0X1.0
R_3_hydroxybehenoyl_ACPs_p
H39.0C22.0O2.0X1.0
trans_delta2_behenoyl_ACPs_p
H37.0C22.0O1.0X1.0
Beheno

### Generate diel leaf model



In [9]:
#Function to make multiple copies of compartments
#args: 1) original cobra model 2) final model 3) number of copies
def multiply_compartments(old_model, new_model, x):
	for y in range(0, x):
		for compartment in old_model.compartments:
			new_model.compartments.update({str(compartment + str(y+1)) : old_model.compartments.get(compartment)})

#Function to make multiple copies of reactions and metabolites
#args: 1) original cobra model 2) final model 3) number of copies
def multiply_rxnmet(old_model, new_model, x):
	from cobra.core import Reaction,Metabolite, Model
	for y in range(0, x):
		for reaction in old_model.reactions:
			newid = str(reaction.id + str(y+1))
			new1 = Reaction(newid)
			new1.name = reaction.name
			new1.subsystem = reaction.subsystem
			new1.lower_bound = reaction.lower_bound
			new1.upper_bound = reaction.upper_bound
			for metabolite in reaction.metabolites:
				newidm = str(metabolite.id + str(y+1))
				newm = Metabolite(newidm)
				newm.name = metabolite.name
				newm.formula = metabolite.formula
				newm.compartment = str(metabolite.compartment + str(y+1))
				newm.charge = metabolite.charge
				new1.add_metabolites({newm : reaction.metabolites.get(metabolite)})
			new_model.add_reaction(new1)

#Function to generate model stats
#args: 1) a cobra model
def check_model(model):
	print "Reactions:"+str(len(model.reactions))
	print "Metabolites:"+str(len(model.metabolites))
	print "Compartments:"+str(len(model.compartments))+"\n"

#Function to add reversible metabolite transfer reactions
#args: 1)a text file with metabolite names 2)a cobra model 3)compartment metaboli-
#-te is transferred from 4)compartment metabolite is transferred to
def allow_transfer(list_doc, model, from_mod, to_mod):
	from cobra.core import Reaction,Metabolite, Model
	a = open(list_doc)
	for line in a:
		b = str(line).replace(" ", "").replace("\n", "").replace("\r", "")
		newid = str("EX_" + b + "_" + str(from_mod) + "_" + str(to_mod))
		new1 = Reaction(newid)
		model.add_reaction(new1)
		new1.name = str("Transfer of" + b + "from" + str(from_mod) + "to" + str(to_mod))
		new1.lower_bound = -1000
		new1.upper_bound = 1000
		new1.add_metabolites({str(b + str(from_mod)) : -1, str(b + str(to_mod)) : 1})

#Function to add metabolite transfer reactions
#args: 1)a text file with metabolite names 2)a cobra model 3)compartment metaboli-
#-te is transferred from 4)compartment metabolite is transferred to
def allow_transfer_oneway(list_doc, model, from_mod, to_mod):
	from cobra.core import Reaction,Metabolite, Model
	a = open(list_doc)
	for line in a:
		b = str(line).replace(" ", "").replace("\n", "").replace("\r", "")
		newid = str("EX_" + b + "_" + str(from_mod) + "_" + str(to_mod) + "ow")
		new1 = Reaction(newid)
		model.add_reaction(new1)
		new1.name = str("Transfer of" + b + "from" + str(from_mod) + "to" + str(to_mod) + "_ow")
		new1.lower_bound = 0
		new1.upper_bound = 1000
		new1.add_metabolites({str(b + str(from_mod)) : -1, str(b + str(to_mod)) : 1}) 




#Function to generate C4 leaf model from a model of plant primary metabolism
#args 1)core model 2)text file listing metabolites allowed to transfer between M and BS 3)text file list-
#-ing amino acid transferred between day and night 4) text file listing non AA metabolite transferred be-
#-tween day and night
#output 1) a C4 model
def generateC4leafmodel(source_model,M_BS_transferlist="met2transfer_M_BS_C4.txt",Day_night_transferlist_AA="met2transfer_AA.txt",Day_night_transferlist_nonAA="met2transfer_nonAA.txt"):
	from cobra.core import Model,Metabolite,Reaction
	# Making C4 model
	C4_model = Model("C4_diel_model")
	
	#Mulitplying compartments, reactions, metabolites
	multiply_compartments(source_model, C4_model, 4)
	multiply_rxnmet(source_model, C4_model, 4)
	
	#creating compartment for dummy metabolites
	C4_model.compartments.update({"f" : "Fake"})
	
	backup_for_C3 = C4_model.copy()
	
	########          SETTING CONSTRAINTS for C4
	#check_model(C4_model)
	
	#Sugar transport
	for i in range(1,5):
		C4_model.reactions.get_by_id(str("Sucrose_tx" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("Sucrose_tx" + str(i))).lower_bound = 0
		C4_model.reactions.get_by_id(str("GLC_tx" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("GLC_tx" + str(i))).lower_bound = 0
	
	#Diel constraints
	for i in (2, 4):
		C4_model.reactions.get_by_id(str("Photon_tx" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("Photon_tx" + str(i))).lower_bound = 0 
	C4_model.reactions.get_by_id(str("Photon_tx1")).upper_bound = C4_model.reactions.get_by_id(str("Photon_tx1")).lower_bound = 600
	C4_model.reactions.get_by_id(str("Photon_tx3")).upper_bound = C4_model.reactions.get_by_id(str("Photon_tx3")).lower_bound = 400
	
	#CO2 constraints
	C4_model.reactions.get_by_id("CO2_tx3").upper_bound = C4_model.reactions.get_by_id("CO2_tx3").lower_bound = 0 
	C4_model.reactions.get_by_id("CO2_tx4").upper_bound = C4_model.reactions.get_by_id("CO2_tx4").lower_bound = 0 
	
	C4_model.reactions.get_by_id("CO2_tx1").lower_bound = 0
	C4_model.reactions.get_by_id("CO2_tx2").upper_bound = 0
	
	#NH4 import
	for i in range(1,5):
		C4_model.reactions.get_by_id(str("NH4_tx" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("NH4_tx" + str(i))).lower_bound = 0
	
	#Nitrate ratio day:night
	met = Metabolite("Nitrate_pseudo")
	met.name = "Nitrate_pseudo"
	met.compartment = "f"
	met.charge = 0
	for i in (1,3):
		C4_model.reactions.get_by_id(str("Nitrate_tx" + str(i))).add_metabolites({met : 2})
	for i in (2,4):
		C4_model.reactions.get_by_id(str("Nitrate_tx" + str(i))).add_metabolites({met : -3})
	
	#Rubisco oxygenase
	for i in range(1,5):
		met = Metabolite("Rubisco_pseudo" + str(i))
		met.name = "Rubisco_pseudo" + str(i)
		met.compartment = "f"
		met.charge = 0
		C4_model.reactions.get_by_id(str("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p" + str(i))).add_metabolites({met : 1})
		C4_model.reactions.get_by_id(str("RXN_961_p" + str(i))).add_metabolites({met : -20})
	
	#Rubsico localisation
	for i in (1,2):
		C4_model.reactions.get_by_id(str("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p" + str(i))).lower_bound = 0
	
	#GDH localisation
	for i in (1,2):
		C4_model.reactions.get_by_id(str("GLYOHMETRANS_RXN_m" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("GLYOHMETRANS_RXN_m" + str(i))).lower_bound = 0
		C4_model.reactions.get_by_id(str("GCVMULTI_RXN_m" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("GCVMULTI_RXN_m" + str(i))).lower_bound =  0
	
	#Maintainence costs
	for i in range(1,5):
		C4_model.reactions.get_by_id(str("ATPase_tx" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("ATPase_tx" + str(i))).lower_bound = 19.5
		met = Metabolite(str("Maint_pseudo" + str(i)))
		met.name = str("Maint_pseudo" + str(i))
		met.compartment = "f"
		met.charge = 0
		C4_model.reactions.get_by_id(str("ATPase_tx" + str(i))).add_metabolites({met : 1})
		C4_model.reactions.get_by_id(str("NADPHoxc_tx" + str(i))).add_metabolites({met : -3})
		C4_model.reactions.get_by_id(str("NADPHoxm_tx" + str(i))).add_metabolites({met : -3})
		C4_model.reactions.get_by_id(str("NADPHoxp_tx" + str(i))).add_metabolites({met : -3})
	
	#plastid enolase constraint
	for i in range(1,5):
		C4_model.reactions.get_by_id(str("2PGADEHYDRAT_RXN_p" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("2PGADEHYDRAT_RXN_p" + str(i))).lower_bound = 0
	
	#plastoquinol oxidase constraint
	for i in range(1,5):
		C4_model.reactions.get_by_id(str("Plastoquinol_Oxidase_p" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("Plastoquinol_Oxidase_p" + str(i))).lower_bound = 0
	
	#Mesophyll starch synthase constraint
	for i in range(1,3):
		C4_model.reactions.get_by_id(str("GLYCOGENSYN_RXN_p" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("GLYCOGENSYN_RXN_p" + str(i))).lower_bound = 0
	
	#plastid hexose transport constraint
	for i in range(1,5):
		C4_model.reactions.get_by_id(str("G6P_Pi_pc" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("G6P_Pi_pc" + str(i))).lower_bound = 0
	
	#switching off PSII
	#for i in (3, 4):
	#	C4_model.reactions.get_by_id(str("PSII_RXN_p" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("PSII_RXN_p" + str(i))).lower_bound = 0
	
	#"biomass"constraint
	#for i in range(1,5):
	#	C4_model.reactions.get_by_id(str("Biomass_tx" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("Biomass_tx" + str(i))).lower_bound = 0
	
	#choice of decarboxylating enzyme
	#for i in (3,4):
		#C4_model.reactions.get_by_id(str("MALIC_NAD_RXN_m" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("MALIC_NAD_RXN_m" + str(i))).lower_bound = 0
		#C4_model.reactions.get_by_id(str("MALIC_NADP_RXN_p" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("MALIC_NADP_RXN_p" + str(i))).lower_bound = 0
		#C4_model.reactions.get_by_id(str("MALIC_NADP_RXN_c" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("MALIC_NADP_RXN_c" + str(i))).lower_bound = 0
		#C4_model.reactions.get_by_id(str("PEPCARBOXYKIN_RXN_c" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("PEPCARBOXYKIN_RXN_c" + str(i))).lower_bound = 0
	
	#metabolite exchanges
	allow_transfer(M_BS_transferlist, C4_model, 1, 3)
	allow_transfer(M_BS_transferlist, C4_model, 2, 4)
	allow_transfer_oneway(Day_night_transferlist_AA, C4_model, 1, 2)
	allow_transfer_oneway(Day_night_transferlist_AA, C4_model, 3, 4)
	allow_transfer(Day_night_transferlist_nonAA, C4_model, 1, 2)
	allow_transfer(Day_night_transferlist_nonAA, C4_model, 3, 4)
	
	# Carbohydrate transfer constraint
	C4_model.reactions.get_by_id("EX_SUCROSE_c_1_3").upper_bound = C4_model.reactions.get_by_id("EX_SUCROSE_c_1_3").lower_bound = 0
	C4_model.reactions.get_by_id("EX_SUCROSE_c_2_4").upper_bound = C4_model.reactions.get_by_id("EX_SUCROSE_c_2_4").lower_bound = 0
	C4_model.reactions.get_by_id("EX_MALTOSE_p_1_3").upper_bound = C4_model.reactions.get_by_id("EX_MALTOSE_p_1_3").lower_bound = 0
	C4_model.reactions.get_by_id("EX_MALTOSE_p_2_4").upper_bound = C4_model.reactions.get_by_id("EX_MALTOSE_p_2_4").lower_bound = 0
	
	# Setting NAD-GAPDH to only run in the direction of glycolysis 
	for i in range(1, 5):
		C4_model.reactions.get_by_id("GAPOXNPHOSPHN_RXN_c"+str(i)).upper_bound = 4000
		C4_model.reactions.get_by_id("GAPOXNPHOSPHN_RXN_c"+str(i)).lower_bound = 0
		C4_model.reactions.get_by_id("GAPOXNPHOSPHN_RXN_p"+str(i)).upper_bound = 4000
		C4_model.reactions.get_by_id("GAPOXNPHOSPHN_RXN_p"+str(i)).lower_bound = 0
	
	#CREATE PHLOEM METABOLIE
	for i in range(3,5):
		met = Metabolite("X_Phloem_contribution_t"+str(i))
		C4_model.reactions.get_by_id("Phloem_output_tx"+str(i)).add_metabolites({met:1})
	
	#setting the total output with 3:1 ratio
	reac = Reaction("Total_Output_tx")
	reac.name = "Total_Output_tx"
	C4_model.add_reaction(reac)
	C4_model.reactions.get_by_id("Total_Output_tx").add_metabolites({"X_Phloem_contribution_t3" : -0.75, "X_Phloem_contribution_t4" : -0.25})
	#print(C4_model.reactions.get_by_id("Total_Output_tx"))
	#check_model(C4_model)
	
	# Increasing model upper bounds
	for rxn in C4_model.reactions:
		if rxn.upper_bound == 1000:
			rxn.upper_bound = 4000
		if rxn.lower_bound == -1000:
			rxn.lower_bound = -4000
	
	# Introducing a continuous apoplast
	for i in range(1, 5):
		rxn = Reaction("H_mc"+str(i))
		rxn.id = "H_mc3"+str(i)
		rxn.upper_bound = 4000
		rxn.lower_bound = -4000
		C4_model.add_reaction(rxn)
		rxn.add_metabolites({"PROTON_c"+str(i) : -1, "PROTON_m"+str(i) : 1})
	
	for i in range(1,5):
		C4_model.reactions.get_by_id("unlProtHYPO_c"+str(i)).upper_bound = C4_model.reactions.get_by_id("unlProtHYPO_c"+str(i)).lower_bound = 0
		rxn = Reaction("H_ec"+str(i))
		rxn.name = "H_ec"+str(i)
		rxn.upper_bound = 4000
		rxn.lower_bound = 0
		C4_model.add_reaction(rxn)
		rxn.add_metabolites({"PROTON_c"+str(i) : 1, "PROTON_e"+str(i) : -1})
		if i == 1:
			for met in C4_model.metabolites.query("_e"+str(i)):
				if not met.id == "Photon_e"+str(i):
					newmet = Metabolite(str(met.id)+"3")
					newmet.name = str(met.id)+"3"
					newmet.compartment = str(met.compartment)+"3"
					newmet.formula = met.formula
					newmet.charge = met.charge
					for rxn in met.reactions:
						j = rxn.metabolites.get(met)
						rxn.add_metabolites({met : -j, newmet : j})
						if str(rxn.id).endswith("_tx"+str(i)):
							if not rxn.id == "Phloem_output_tx1":
								if not rxn.id == "CO2_tx1":
									newid = str(rxn.id + "3")
									rxn.id = newid
		if i == 3:
			for met in C4_model.metabolites.query("_e"+str(i)):
				if not met.id == "Photon_e"+str(i):
					for newmet in C4_model.metabolites.query("_e13"):
						metstart = str(met.id).split("_")[0]
						if str(newmet.id).startswith(metstart):
							for rxn in met.reactions:
								j = rxn.metabolites.get(met)
								rxn.add_metabolites({met : -j, newmet : j})
								if str(rxn.id).endswith("_tx"+str(i)):
									if not rxn.id == "Phloem_output_tx3":
										if not rxn.id == "CO2_tx3":
											rxn.upper_bound = rxn.lower_bound = 0
		if i == 2:                    
			for met in C4_model.metabolites.query("_e"+str(i)):
				if not met.id == "Photon_e"+str(i):
					newmet = Metabolite(str(met.id)+"4")
					newmet.name = str(met.id)+"4"
					newmet.compartment = str(met.compartment)+"4"
					newmet.formula = met.formula
					newmet.charge = met.charge
					for rxn in met.reactions:
						j = rxn.metabolites.get(met)
						rxn.add_metabolites({met : -j, newmet : j})
						if str(rxn.id).endswith("_tx"+str(i)):
							if not rxn.id == "Phloem_output_tx2":
								if not rxn.id == "CO2_tx2":
									newid = str(rxn.id + "4")
									rxn.id = newid
		if i == 4:
			for met in C4_model.metabolites.query("_e"+str(i)):
				if not met.id == "Photon_e"+str(i):
					for newmet in C4_model.metabolites.query("_e24"):
						metstart = str(met.id).split("_")[0]
						if str(newmet.id).startswith(metstart):
							for rxn in met.reactions:
								j = rxn.metabolites.get(met)
								rxn.add_metabolites({met : -j, newmet : j})
								if str(rxn.id).endswith("_tx"+str(i)):
									if not rxn.id == "Phloem_output_tx4":
										if not rxn.id == "CO2_tx4":
											rxn.upper_bound = rxn.lower_bound = 0
	
	#Restoring the previous constraints for CO2
	C4_model.reactions.get_by_id("CO2_ec3").upper_bound = C4_model.reactions.get_by_id("CO2_ec3").lower_bound = 0 
	C4_model.reactions.get_by_id("CO2_ec4").upper_bound = C4_model.reactions.get_by_id("CO2_ec4").lower_bound = 0 
	
	C4_model.reactions.get_by_id("CO2_ec1").lower_bound = 0
	C4_model.reactions.get_by_id("CO2_ec2").upper_bound = 0
	
	# Optimising...
	C4_model.objective = C4_model.reactions.get_by_id("Total_Output_tx")
	C4_model.reactions.get_by_id("Total_Output_tx").objective_value = 1
	return C4_model


In [10]:
leaf_model = model.copy()

leaf_model = generateC4leafmodel(leaf_model,M_BS_transferlist="met2transfer_M_BS_C4.txt",Day_night_transferlist_AA="met2transfer_AA.txt",Day_night_transferlist_nonAA="met2transfer_nonAA.txt")

leaf_model.reactions.Total_Output_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(leaf_model)
print("Phloem output = ")
print(leaf_model.reactions.Total_Output_tx.flux)

Phloem output = 
7.18818790078


In [11]:
leaf_model = model.copy()


k = "leaf"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({leaf_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])/leaf_model.metabolites.get_by_id(met).formula_weight})
        RXN2.add_metabolites({leaf_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])/leaf_model.metabolites.get_by_id(met).formula_weight})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k])/leaf_model.metabolites.get_by_id(met).formula_weight)
if tot==0:
    RXN1.add_metabolites({leaf_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({leaf_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({leaf_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 0
leaf_model.add_reaction(RXN1)
    
RXN2.add_metabolites({leaf_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 0
leaf_model.add_reaction(RXN2)


generateMissingFormula(leaf_model)

rxn = Reaction("Biomass_leaf_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({leaf_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met]["stem"])/leaf_model.metabolites.get_by_id(met).formula_weight})
rxn.lower_bound = 0
rxn.upper_bound = 1000
leaf_model.add_reaction(rxn)

leaf_model_sink = generateC4leafmodel(leaf_model,M_BS_transferlist="met2transfer_M_BS_C4.txt",Day_night_transferlist_AA="met2transfer_AA.txt",Day_night_transferlist_nonAA="met2transfer_nonAA.txt")


met = Metabolite("X_Biomass_contribution_day")
leaf_model_sink.reactions.Biomass_leaf_tx1.add_metabolites({met:1})
leaf_model_sink.reactions.Biomass_leaf_tx3.add_metabolites({met:1})
leaf_model_sink.metabolites.X_Phloem_contribution_t3.remove_from_model()
leaf_model_sink.reactions.Total_Output_tx.add_metabolites({met:-3})

met = Metabolite("X_Biomass_contribution_night")
leaf_model_sink.reactions.Biomass_leaf_tx2.add_metabolites({met:1})
leaf_model_sink.reactions.Biomass_leaf_tx4.add_metabolites({met:1})
leaf_model_sink.metabolites.X_Phloem_contribution_t4.remove_from_model()
leaf_model_sink.reactions.Total_Output_tx.add_metabolites({met:-1})

leaf_model_sink.reactions.Total_Output_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(leaf_model_sink)
print("Biomass accumulation = ")
print(leaf_model_sink.reactions.Total_Output_tx.flux)

Loop = 1
Photon_e

Fatty_acyl_ACP_p
H31.0C16.0O1.0X1.0
Protein_polymerisation_cost_c

Protein_processing_cost_c

Photon_p

Palmitate_b
H31.0C16.0O2.0
Photon_p

Protein_tranlocation_cost_c

Long_Chain_Acyl_CoAs_p
C37.0H66.0O17.0N7.0P3.0S1.0
Fatty_Acids_c
H31.0C16.0O2.0
Fatty_Acids_p
H31.0C16.0O2.0
3_oxo_stearoyl_ACPs_p
H33.0C18.0O2.0X1.0
R_3_hydroxystearoyl_ACPs_p
H35.0C18.0O2.0X1.0
Octadec_2_enoyl_ACPs_p
H33.0C18.0O1.0X1.0
Stearoyl_ACPs_p
H33.0C18.0X1.0
Palmitoleoyl_ACP_p
X1.0C16.0O1.0H31.0
hexadecadienoate_ACP_p
X1.0C16.0O1.0H31.0
hexadecatrienoate_ACP_p
X1.0C16.0O1.0H31.0
Oleoyl_ACPs_p
X1.0C18.0H33.0
Octadecadienoyl_ACP_p
X1.0C18.0H33.0
Octadecatrienoyl_ACP_p
X1.0C18.0H33.0
3_oxo_arachidoyl_ACPs_p
H35.0C20.0O1.0X1.0
R_3_hydroxyarachidoyl_ACPs_p
H37.0C20.0O1.0X1.0
trans_delta2_arachidoyl_ACPs_p
H35.0C20.0X1.0
Arachidoyl_ACPs_p
H35.0C20.0O1.0X1.0
3_oxo_behenoyl_ACPs_p
H37.0C22.0O2.0X1.0
R_3_hydroxybehenoyl_ACPs_p
H39.0C22.0O2.0X1.0
trans_delta2_behenoyl_ACPs_p
H37.0C22.0O1.0X1.0
Beheno

## Transcriptomics

In [12]:
import pythoncyc
from Functions import *

cyc = pythoncyc.select_organism("corn")

In [13]:
model = addGPR2Models(model,cyc)

--------------
This list of metabolic reactions are ignored
['RXN_9650_p', '2_KETO_ADIPATE_DEHYDROG_RXN_m', 'Phytol_biosynthesis_p', 'CYSTEINE_AMINOTRANSFERASE_RXN_m', 'GLYCINE_TRNA_LIGASE_RXN_c', 'RXN66_1_c', 'RXN_9648_p', 'RXN-9651', 'Plastidial_ATP_Synthase_p', 'GGPP_biosynthesis_p', 'RXN_9653_p', 'lycopene_biosynthesis_p', 'RXN_2141_p', 'SUCCINYL_COA_HYDROLASE_RXN_m', 'PROTON_ATPase_c', 'MDA_Fd_Ascorbate_p', 'MercaptoPyruvateSulfurtransferase_m', 'Phytol_degradation_p', 'RXN_9652_p', 'A_B_oxidation_x', 'unlProtHYPO_c', 'Mitochondrial_ATP_Synthase_m', 'IPP_biosynthesis_c', 'Mehler_Reaction_p', 'Beta_Oxidation_x', 'HMBPP_synthesis_p', 'OROTATE_REDUCTASE_NADH_RXN_p', 'Ferredoxin_Plastoquinone_Reductase_p', 'RXN_9651_p', 'NADPH_Dehydrogenase_p', 'Plastoquinol_Oxidase_p', 'SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m', 'RXN_1781_v', 'PREPHENATE_DEHYDROGENASE_NADP_RXN_p', 'PREPHENATEDEHYDROG_RXN_p', 'MALEYLACETOACETATE_ISOMERASE_RXN_c', 'RXN_9654_p', 'LCYSDESULF_RXN_c', 'RXN_9958_NAD_m', 'HEXO

cobra/core/reaction.py:446 UserWarning: malformed gene_reaction_rule '(() or (Zm00001d018021) or (Zm00001d037501) or (Zm00001d039777) or (Zm00001d036858) or (Zm00001d043086) or (Zm00001d023880) or (Zm00001d012518) or (Zm00001d029025) or (Zm00001d041697))' for <Reaction 3PGAREARR_RXN_p at 0x7fd70aa2f250>
cobra/core/reaction.py:446 UserWarning: malformed gene_reaction_rule '(() or (Zm00001d018021) or (Zm00001d037501) or (Zm00001d039777) or (Zm00001d036858) or (Zm00001d043086) or (Zm00001d023880) or (Zm00001d012518) or (Zm00001d029025) or (Zm00001d041697))' for <Reaction 3PGAREARR_RXN_c at 0x7fd70a8ebb90>


In [14]:
len(model.genes)

1578